Stage repeatability

In [1]:
from pathlib import Path
import os
import nd2
import numpy as np
import napari
import pandas as pd
from utils import plots_results_on_image, calculate_colocalization, calculate_props, extract_labels, save_bar_graph
import plotly.express as px

In [5]:
# Define the raw image path
raw_img_path = Path('./raw_data/20200703_20X_stage_repeatability/crosses_20x_green.nd2')

# Construct the denoised image path using pathlib
denoised_img_path = raw_img_path.with_name(f'{raw_img_path.stem} - Denoised.nd2')

# Read raw and denoised images
raw_img = nd2.imread(raw_img_path)
denoised_img = nd2.imread(denoised_img_path)

# Read the file and extract the metadata
f = nd2.ND2File(raw_img_path)
metadata = f.metadata

# Extract and print the required metadata
for channel in metadata.channels:
    channel_meta = channel.channel
    microscope = channel.microscope
    volume = channel.volume
    print(f"Channel Name: {channel_meta.name}")
    print(f"Index: {channel_meta.index}")
    print(f"Emission Lambda: {channel_meta.emissionLambdaNm}")
    print(f"Excitation Lambda: {channel_meta.excitationLambdaNm}")
    print(f"Objective Name: {microscope.objectiveName}")
    print(f"Modality Flags: {', '.join(microscope.modalityFlags)}")
    print(f'XYZ_pixel_size: {volume.axesCalibration}')
    print()  # Add a blank line for readability

# Print JSON info
print(f.text_info)
# Close file for garbage collection
f.close()

Channel Name: AR_green
Index: 0
Emission Lambda: 511.0
Excitation Lambda: 477.0
Objective Name: S Plan Fluor LWD 20xC
Modality Flags: fluorescence, spinningDiskConfocal
XYZ_pixel_size: (0.330313846974365, 0.330313846974365, 1.0)

{'description': 'Metadata:\r\nDimensions: T(109) x XY(6) x λ(1)\r\nCamera Name: Kinetix, A24B723015\r\nNumerical Aperture: 0.7\r\nRefractive Index: 1\r\n Name: AR_green\r\n Component Count: 1\r\n Relay Lens Magnification: 1x\r\n Modality: Widefield Fluorescence, Spinning Disk Confocal\r\n Camera Settings:   Binning: 1x1\r\n  Bit Depth: 12-bit\r\n  Exposure: 150 ms\r\n  Readout Speed: 100.0MHz\r\n  Multiplier: Off\r\n  Camera Mode: 12-bit (CMS)\r\n  Temperature: 0.0°C\r\n  \r\n  Conversion Gain: Sensitivity\r\n  Denoise.ai OFF\r\n  Clarify.ai OFF\r\n  Denoising: Off\r\n  Fan Speed: High\r\n\r\n Microscope Settings:   Microscope: Ti2 Microscope\r\n  DIC Prism, position: In\r\n  Nikon Ti2, FilterChanger(Turret-Lo): 1 (Empty)\r\n  LightPath: L100\r\n  PFS, mirror:

In [4]:
metadata

Metadata(contents=Contents(channelCount=1, frameCount=654), channels=[Channel(channel=ChannelMeta(name='AR_green', index=0, color=Color(r=0, g=255, b=0, a=1.0), emissionLambdaNm=511.0, excitationLambdaNm=477.0), loops=LoopIndices(NETimeLoop=None, TimeLoop=0, XYPosLoop=1, ZStackLoop=None), microscope=Microscope(objectiveMagnification=20.0, objectiveName='S Plan Fluor LWD 20xC', objectiveNumericalAperture=0.7, zoomMagnification=1.0, immersionRefractiveIndex=1.0, projectiveMagnification=None, pinholeDiameterUm=50.0, modalityFlags=['fluorescence', 'spinningDiskConfocal']), volume=Volume(axesCalibrated=(True, True, False), axesCalibration=(0.330313846974365, 0.330313846974365, 1.0), axesInterpretation=('distance', 'distance', 'distance'), bitsPerComponentInMemory=16, bitsPerComponentSignificant=12, cameraTransformationMatrix=(-0.9999965534187834, 0.0026254810138932397, -0.0026254810138932397, -0.9999965534187834), componentCount=1, componentDataType='unsigned', voxelCount=(512, 512, 1), com

In [3]:
# Visualize results in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(denoised_img)

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


<Image layer 'denoised_img' at 0x24b445f86a0>